In [ ]:
import json
import safetensors
import safetensors.torch
from transformers import AutoConfig

In [ ]:
dora_weights = safetensors.torch.load_file("/workspace/models/Meta-Llama-3-1-405B-Instruct-4bit-DoRA/step_500/model_state_dict.safetensors")

In [ ]:
config_dict = json.load(open("/workspace/models/Meta-Llama-3-1-405B-Instruct-4bit-DoRA/step_500/config.json"))["qlora_config"] 

In [ ]:
config = AutoConfig.from_pretrained("meta-llama/Meta-Llama-3.1-405B-Instruct")

In [ ]:
16384/(128/8)

1024.0

In [ ]:
config

LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3.1-405B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 16384,
  "initializer_range": 0.02,
  "intermediate_size": 53248,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 128,
  "num_hidden_layers": 126,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.0",
  "use_cache": true,
  "vocab_size": 128256
}

In [ ]:
config_dict

{'lora_target_modules': ['q_proj',
  'k_proj',
  'v_proj',
  'o_proj',
  'gate_proj',
  'up_proj',
  'down_proj'],
 'compute_dtype': 'bfloat16',
 'lora_rank': 64,
 'layer_nbits': {'q_proj': 4,
  'k_proj': 4,
  'v_proj': 4,
  'o_proj': 4,
  'gate_proj': 4,
  'up_proj': 4,
  'down_proj': 4},
 'layer_groupsizes': {'q_proj': 128,
  'k_proj': 128,
  'v_proj': 128,
  'o_proj': 128,
  'gate_proj': 128,
  'up_proj': 128,
  'down_proj': 128}}

In [ ]:
for k,v in dora_weights.items():
    if k.split(".")[2] in "0 1 2 3 4 5".split():
	    print(k, v.shape)

model.layers.0.mlp.down_proj.dora_layer.lora_A.weight torch.Size([64, 53248])
model.layers.0.mlp.down_proj.dora_layer.lora_B.weight torch.Size([16384, 64])
model.layers.0.mlp.down_proj.magnitude_layer.magnitude torch.Size([16384])
model.layers.0.mlp.gate_proj.dora_layer.lora_A.weight torch.Size([64, 16384])
model.layers.0.mlp.gate_proj.dora_layer.lora_B.weight torch.Size([53248, 64])
model.layers.0.mlp.gate_proj.magnitude_layer.magnitude torch.Size([53248])
model.layers.0.mlp.up_proj.dora_layer.lora_A.weight torch.Size([64, 16384])
model.layers.0.mlp.up_proj.dora_layer.lora_B.weight torch.Size([53248, 64])
model.layers.0.mlp.up_proj.magnitude_layer.magnitude torch.Size([53248])
model.layers.0.self_attn.k_proj.dora_layer.lora_A.weight torch.Size([64, 16384])
model.layers.0.self_attn.k_proj.dora_layer.lora_B.weight torch.Size([2048, 64])
model.layers.0.self_attn.k_proj.magnitude_layer.magnitude torch.Size([2048])
model.layers.0.self_attn.o_proj.dora_layer.lora_A.weight torch.Size([64, 16

In [ ]:
w =dora_weights['model.layers.0.self_attn.k_proj.dora_layer.lora_B.weight']

In [ ]:
w.shape

torch.Size([2048, 64])

In [ ]:
w[:1024]

tensor([[-1.8328e-06,  7.8082e-06, -8.2254e-06,  ...,  4.2319e-06,
          2.1309e-06, -1.0878e-06],
        [ 8.3074e-07, -2.0489e-07, -1.2014e-07,  ...,  1.1772e-06,
          1.5870e-06, -1.1781e-07],
        [-7.8231e-07,  3.1199e-08,  1.1921e-06,  ...,  2.0862e-07,
         -2.9989e-07,  2.7195e-07],
        ...,
        [-6.2957e-07,  5.3048e-06, -7.8678e-06,  ...,  4.6194e-06,
          2.1756e-06,  8.0559e-08],
        [ 8.4639e-06, -3.3855e-05,  7.6294e-05,  ..., -3.4571e-05,
         -1.0133e-05,  7.0408e-07],
        [ 1.8179e-06,  3.5912e-06, -1.1146e-05,  ...,  5.6922e-06,
          2.7716e-06,  1.0505e-06]], dtype=torch.bfloat16)

In [ ]:
w[1024:]

tensor([[-4.0054e-05,  1.4973e-04, -2.2602e-04,  ...,  1.6117e-04,
          9.1553e-05, -2.1219e-05],
        [ 3.5286e-05, -5.1737e-05,  7.2956e-05,  ..., -4.7922e-05,
         -1.9670e-05, -8.8215e-06],
        [-6.1989e-05,  8.4877e-05, -8.6308e-05,  ...,  5.7459e-05,
          4.2439e-05, -2.7776e-05],
        ...,
        [-3.0398e-06, -5.6326e-06,  6.4850e-05,  ..., -3.1233e-05,
         -8.1584e-07, -1.0610e-05],
        [-2.9564e-05,  1.8239e-05, -7.8201e-05,  ...,  2.4676e-05,
         -1.2398e-05,  2.8312e-06],
        [-2.6464e-05,  2.8372e-05, -7.4387e-05,  ...,  2.7895e-05,
         -4.9174e-06, -2.2054e-06]], dtype=torch.bfloat16)